In [1]:
%load_ext autoreload
%autoreload 2

In [29]:
from sherpa_ai.memory.belief import Belief
from sherpa_ai.memory.state_machine import SherpaStateMachine
from transitions.extensions import HierarchicalGraphMachine
from sherpa_ai.models import SherpaChatOpenAI
from sherpa_ai.policies.react_policy import ReactPolicy
from sherpa_ai.agents.qa_agent import QAAgent
from sherpa_ai.events import Event, EventType

In [ ]:
from sherpa_ai.actions.base import BaseAction

class HelloWorld(BaseAction):
    name: str = "print_hello_world"
    args: dict = {"name": "the name of the person doing hello world (str)"}
    usage: str = "print hello world given a name"

    def execute(self, name: str):
        print(f"Hello World from {name}")
        return f"{name}"

class Addition(BaseAction):
    name: str = "perform_addition"
    args: dict = {"num1": "the first number (int)", "num2": "the second number (int)"}
    usage: str = "Add two numbers together"

    def execute(self, num1: int, num2: int):
        result = num1 + num2
        print(f"Result: {result} for {self.belief.get('print_hello_world')}")
        return result

In [18]:
belief = Belief()
hello_action = HelloWorld(usage="performing hello world", belief=belief)
addition_action = Addition(belief=belief)

action_map = {
    hello_action.name: hello_action,
    addition_action.name: addition_action
}

In [19]:
states = ["HelloWorld", "Calculate", "Done"]
initial = "HelloWorld"
transitions = [{
    "trigger": "do_hello_world",
    "source": "HelloWorld",
    "dest": "Calculate",
    "before": "print_hello_world",
},
{
    "trigger": "do_nothing",
    "source": "HelloWorld",
    "dest": "Done",
},
{
    "trigger": "add_numbers",
    "source": "Calculate",
    "dest": "Done",
    "before": "perform_addition"
}]

In [20]:
sm = SherpaStateMachine(
    states=states,
    transitions=transitions,
    initial=initial,
    action_map=action_map,
    sm_cls=HierarchicalGraphMachine,
)
belief.state_machine = sm
print(sm.sm.get_graph().draw(None))

In [21]:
llm = SherpaChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
policy = ReactPolicy(
    role_description="Help the user finish the task",
    output_instruction="Determine which action and arguments would be the best continuing the task",
    llm=llm,
)

In [22]:
belief.get_actions()

In [23]:
qa_agent = QAAgent(llm=llm, belief=belief, num_runs=3, policy=policy)

In [24]:
belief.set_current_task(Event(EventType.task, "user", "User is Gunter. 10. 20."))

In [25]:
print(qa_agent.run())

In [26]:
[str(event) for event in belief.internal_events]

In [29]:
from sherpa_ai.memory.belief import Belief
from sherpa_ai.memory.state_machine import SherpaStateMachine
from transitions.extensions import HierarchicalGraphMachine
from sherpa_ai.models import SherpaChatOpenAI
from sherpa_ai.policies.react_policy import ReactPolicy
from sherpa_ai.agents.qa_agent import QAAgent
from sherpa_ai.events import Event, EventType

In [18]:
belief = Belief()
hello_action = HelloWorld(usage="performing hello world", belief=belief)
addition_action = Addition(belief=belief)

action_map = {
    hello_action.name: hello_action,
    addition_action.name: addition_action
}

In [20]:
sm = SherpaStateMachine(
    states=states,
    transitions=transitions,
    initial=initial,
    action_map=action_map,
    sm_cls=HierarchicalGraphMachine,
)
belief.state_machine = sm
print(sm.sm.get_graph().draw(None))

---
State Machine
---
stateDiagram-v2
  direction LR
  classDef s_default fill:white,color:black
  classDef s_inactive fill:white,color:black
  classDef s_parallel color:black,fill:white
  classDef s_active color:red,fill:darksalmon
  classDef s_previous color:blue,fill:azure
  
  state "HelloWorld" as HelloWorld
  Class HelloWorld s_active
  state "Calculate" as Calculate
  Class Calculate s_default
  state "Done" as Done
  Class Done s_default
  
  HelloWorld --> Calculate: do_hello_world
  HelloWorld --> Done: do_nothing
  Calculate --> Done: add_numbers
  [*] --> HelloWorld


In [22]:
belief.get_actions()

[{
     "name": "do_hello_world",
     "args": {
         "name": "the name of the person doing hello world (str)"
     },
     "usage": "performing hello world Transit the state from HelloWorld to Calculate"
 },
 {
     "name": "do_nothing",
     "args": {},
     "usage": "do_nothing Transit the state from HelloWorld to Done"
 }]

In [24]:
belief.set_current_task(Event(EventType.task, "user", "User is Gunter. 10. 20."))

In [26]:
[str(event) for event in belief.internal_events]

["do_hello_world: EventType.action - Action: do_hello_world starts, Args: {'name': 'Gunter'}",
 'do_hello_world: EventType.action_output - Action: do_hello_world finishes, Observation: Gunter',
 "add_numbers: EventType.action - Action: add_numbers starts, Args: {'num1': 10, 'num2': 20}",
 'add_numbers: EventType.action_output - Action: add_numbers finishes, Observation: 30']

In [27]:
belief.dict

{'print_hello_world': 'Gunter', 'perform_addition': 30}

In [28]:
belief.get_actions()

[]

In [60]:
belief.state_machine.state

'HelloWorld'